# Image Recognition

### Data
- https://en.wikipedia.org/wiki/CIFAR-10

### Libraries
- http://www.numpy.org/
- https://keras.io/
- https://www.tensorflow.org/

### Source
- https://www.linkedin.com/learning/deep-learning-image-recognition

In [21]:
# Libraries

import keras
from keras.datasets import cifar10
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image

import numpy as np
from pathlib import Path

In [11]:
# Load data set

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [12]:
# Build a model and add layers
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [13]:
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("models/image_recognition_model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("models/image_recognition_model_weights.h5")

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 180s 4ms/step - loss: 1.5549 - acc: 0.4360 - val_loss: 1.1768 - val_acc: 0.5751
Epoch 2/30
50000/50000 [==============================] - 190s 4ms/step - loss: 1.1340 - acc: 0.5969 - val_loss: 0.9840 - val_acc: 0.6483
Epoch 3/30
50000/50000 [==============================] - 196s 4ms/step - loss: 0.9581 - acc: 0.6636 - val_loss: 0.8591 - val_acc: 0.7019
Epoch 4/30
50000/50000 [==============================] - 190s 4ms/step - loss: 0.8504 - acc: 0.7034 - val_loss: 0.7877 - val_acc: 0.7269
Epoch 5/30
50000/50000 [==============================] - 187s 4ms/step - loss: 0.7769 - acc: 0.7269 - val_loss: 0.7333 - val_acc: 0.7507
Epoch 6/30
50000/50000 [==============================] - 179s 4ms/step - loss: 0.7258 - acc: 0.7445 - val_loss: 0.7023 - val_acc: 0.7603
Epoch 7/30
50000/50000 [==============================] - 184s 4ms/step - loss: 0.6824 - acc: 0.7612 - val_loss: 0.6863 - 

In [14]:
# These are the CIFAR10 class labels from the training data (in order from 0 to 9)
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

In [17]:
# Load the model for prediction

# Load the json file that contains the model's structure
f = Path("models/image_recognition_model_structure.json")
prediction_model_structure = f.read_text()

# Recreate the Keras model object from the json data
prediction_model = model_from_json(prediction_model_structure)

# Re-load the model's trained weights
prediction_model.load_weights("models/image_recognition_model_weights.h5")

In [32]:
# Predict

# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("images/car.png", target_size=(32, 32))

# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)

# Make a prediction using the model
results = prediction_model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]
print(single_result)

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

# Print the result
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
This is image is a Car - Likelihood: 1.000000
